<a href="https://colab.research.google.com/github/pratikb2805/Ml-Library-I/blob/master/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import warnings
np.seterr(divide='ignore', invalid='ignore')

class layers:
    def __init__(self,neu,r,c):
        self.neu=int(neu)
        self.wt=np.random.randn(r,c)
        self.bias=np.random.randn(self.neu)
        

class network:
    def __init__(self,data_in,data_out,layers_list,l_rate=0.05):
        self.x=np.array(data_in)
        self.y=np.array(data_out)    
        xx=np.array_split(self.x.T,2)
        self.xt=xx[0].T
        self.xc=xx[1].T
        yy=np.array_split(self.y.T,2)
        self.yt=yy[0].T
        self.yc=yy[1].T 
        self.l=int(len(li))
        self.ly=layers_list
        self.l_rate= float(l_rate)
        try:
            self.n=self.xt.shape[1]
        except:
            self.n=self.xt.shape[0]
        self.m=int(self.n/100)
        self.x_split=np.array_split(self.xt.T,self.m)
        self.y_split=np.array_split(self.yt.T,self.m)
     
    def sigm(self,x):
        try:
            return 1/(1+np.e**-x)
        except:
            return (1+np.abs(x)/x)/2


    def der_sigm(self,x):
        return sigm(x)/(1-sigm(x))

    def forward(self):
        for i in range(self.l):
            self.ly[i].z=np.empty((self.ly[i].neu,self.xt.shape[1]),dtype=np.float64)
            self.ly[i].a=np.empty((self.ly[i].neu,self.xt.shape[1]),dtype=np.float64)

        for i in range(self.l):
            if i==0:
                for j in range(int(self.ly[i].neu)):
                    self.ly[i].z[j,:]=np.dot(self.ly[i].wt,self.xt)[j,:] +self.ly[i].bias[j]              
            else:
                for j in range(int(self.ly[i].neu)):
                    self.ly[i].z[j,:]=np.dot(self.ly[i].wt,self.ly[i-1].a)[j,:] + self.ly[i].bias[j]
            self.ly[i].a=self.sigm(self.ly[i].z)
    
    def cost(self):
        return np.sum((np.dot(self.yt,np.log(self.ly[-1].a).T))+ np.dot( 1-self.yt,np.log(1-self.ly[-1].a).T ))
    
    def backprop(self):
        for i in range(self.l-1, -1, -1):
            if i!=self.l-1:
                self.ly[i].error=np.dot(self.ly[i+1].wt.T, self.ly[i+1].a)
            else:
                self.ly[i].error=self.ly[i].a-self.yt
    
    def update(self):
        for i in range(self.l-1, -1, -1):
            if i!=0:
                temp= np.dot(self.ly[i].error,self.ly[i-1].a.T)
            else:
                temp= np.dot(self.ly[i].error,self.xt.T)            
            temp_bias= np.mean(self.ly[i].error, axis=1)
            self.ly[i].wt=self.ly[i].wt-temp*self.l_rate
            self.ly[i].bias=self.ly[i].bias-np.ndarray.flatten(temp_bias)*self.l_rate

    def train(self,epoch):
        try:
            epoch=int(int(epoch)/self.m)
            r=int(self.m)
        except:
            epoch=int(epoch)
            r=int(1)
        for i in range(int(epoch)):
            for j in range(int(r)):
                try:
                    self.xt=self.x_split[i].T
                    self.yt=self.y_split[i].T
                    self.forward()
                    self.backprop()
                    self.update()
                except:
                    pass

    def implement(self,x_data):            # just one sample at a time
        x_data=np.array(x_data)/100
        l=int(x_data.shape[0])
        x_data=np.expand_dims(x_data,0)
        x_data=x_data.T
        self.xi=x_data
        for i in range(self.l):
            if i==0:
                self.ly[i].z=np.dot(self.ly[i].wt,x_data)+self.ly[i].bias
                self.ly[i].a=self.sigm( self.ly[i].z)
            else:
                self.ly[i].z=np.dot(self.ly[i].wt,self.ly[i-1].a)+self.ly[i].bias
                self.ly[i].a=self.sigm(self.ly[i].z)
        print(self.ly[-1].a)

    def train2(self):
        self.forward()
        self.backprop()
        self.update()
        while np.mean(np.abs(self.ly[-1].error))>10**-2:
            self.forward()
            self.backprop()
            self.update()




In [0]:

data=pd.read_csv("/content/sample_data/mnist_train_small.csv",header=None)
data.dropna()
d=np.array(data)
y=d[:,[0]]
x=np.delete(d,0,1)/510
z=np.eye(10)
yy=np.empty((y.shape[0],10),dtype=np.float64)
for i in range(y.shape[0]):
    for j in range(10):
        if y[i]==j:
            yy[i,:]=z[j,:]
        else:
            pass

In [0]:
li=[]
li.append(layers(10,10,784))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))
li.append(layers(10,10,10))



In [0]:
model=network(x.T,yy.T,li)


In [0]:
model.forward()


In [0]:
model.backprop()

In [0]:
model.update()

In [0]:
model.train2()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: overflow encountered in power


In [35]:
np.mean(model.ly[-1].error)

0.20900572622887625

In [0]:
data=np.loadtxt("/content/Outlier.txt", delimiter=',', dtype=np.float64)

In [0]:
x=data[:,[0,1]]

In [0]:
x=x/30

In [0]:
y=data[:,-1]

In [0]:
l=[]
for i in range(2):
    l.append(model.ly[i].wt)


In [0]:
for i in range(2):
    model.ly[i].wt=np.array(l[i])